**What is this notebook?**

We will use this notebook to 
* visualize longitudinal profile response to changing boundary conditions
* illustrate static, lithologic knickpoints in longitudinal profiles
* explore the effects of changing bedrock erodibility on longitudinal profile evolution and knickpoint migration rates.

The model simulates the evolution of detachment-limited channels in an actively uplifting landscape. The landscape evolves according to the **stream power equation**:

\begin{equation}
 \frac{d z}{d t} = -K A^m S^n + U
\end{equation}
Here, $K$ is the erodibility coefficient on fluvial incision, which is thought to be positively correlated with climate wetness, or storminess (this is hard to quantify) and to be negatively correlated with rock strength (again, rock strength is hard to quantify). $m$ and $n$ are positive exponents, usually thought to have a ratio, $m/n \approx 0.5$. $A$ is drainage area and $S$ is the slope of steepest descent ($-\frac{dz}{dx}$) where $x$ is horizontal distance (positive in the downslope direction) and $z$ is elevation. (If slope is negative there is no fluvial erosion.) $U$ is an externally-applied rock uplift field.

The fluvial erosion term is also known as the stream power equation. Before using this notebook you should be familiar with this equation from class lectures and reading. 

For a great overview of the stream power equation, see: 

- Whipple and Tucker, 1999, Dynamics of the stream-power river incision model: Implications for height limits of mountain ranges, landscape response timescales, and research needs, Journal of Geophysical Research.

For some great illustrations of modeling with the stream power equation, see:

- Tucker and Whipple, 2002, Topographic outcomes predicted by stream erosion models: Sensitivity analysis and intermodel comparison, Journal of Geophysical Research.

Helpful background on landscape sensitivity to rock uplift rates and patterns can be found here:

- Kirby and Whipple, 2012, Expression of active tectonics in erosional landscapes, Journal of Structural Geology.

**Now on to the code...**

First we have to import the python modules that are needed to run this code. You will probably have to install a new python module called celluloid. 

* Run the cell below only once.
* Then restart the kernel and
* comment out the line in the cell below.

In [ ]:
#!pip install celluloid

In the cell below, import python modules.

In [ ]:
#%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera

**In the cell below** we define a function that runs the model with inputs we define below. **Do run this cell,** but don't change anything in this cell.

In [ ]:
def runmodel(tmax, z, num_plots):
    fig = plt.figure(fignum, figsize = (8,7))
    plt.title(title_str, fontsize = 14)
    plt.grid()
    camera = Camera(fig)
    
    dt = 100
    next_plot=1 
    next_plot_animate = 1
    plot_int=1e4
    plot_int=tmax/num_plots

    plot_int_animate = tmax/8
    z0=z.copy()       #elevations to change
    for t in np.arange(1,tmax+dt,dt):
        #calc new slopes
        S[:-1]=-np.diff(z)/dx
        S[-1]= S[-2]
        #calculate erosion using stream power model. A = ka*x**h
        E = (ka*x**h)**m * S**n * Ki
        # calculate change in z over time step
        dzdt=(U - E)
        #update height of the landscape
        z += (dzdt*dt)        
        #boundary condition, base level defined in each case
        z[-1]=bl  # boundary condition is fixed during model      
        #calcs for finding knickpoint
        curvature=np.diff(S)/dx      #curvature of long profile
        kk=np.sign(curvature)        #sign of curvature
        ll=np.where(kk>0)[0]      #Find where sign of curvature is positive (a convexity)    
        if np.size(ll) >0:
            kppind = np.argmax(curvature[ll])
            kpp=ll[kppind-1]                        # NEW VERSION, unsure if it's better
            #^ this version above is the best. have to have -1 in there to adjust index
            # that was found on the kp array to the x array, which is one element longer
        else:
            kpp = 0    #if there is no KP, assign value of 0 to kpp for plotting below

        if t>=next_plot_animate:
            plt.figure(fignum)
            ########animated profile, plot profile at time step
            #below inital profile
            plt.plot(x,z0, color="forestgreen", linestyle="--")  
            i_line = plt.plot(x,z, color = "orchid", linewidth = 3)
            if lithkp:
                plt.plot(x[K_idx_start:K_idx_end], z[K_idx_start:K_idx_end],fillstyle="none",markeredgecolor="k", marker=".",ls = " ", label="less erodible")
            #include dots for knickpoints
            plt.scatter(x[kpp],z[kpp], marker='o', s = 50, c = "steelblue", zorder=99, label="kpp, t = "+str(round(t/1000))+"ky")
            plt.xlabel('distance from source (m)',fontsize = 14)
            plt.ylabel('elevation (m)',fontsize = 14)
            plt.xlim([0, 3e4])    
            plt.ylim([bl,max(z0)])
            i = round(t/1000)
            plt.legend(i_line, [f'time = {i}ky'])
            plt.tight_layout()
            camera.snap()
            next_plot_animate += plot_int_animate
    
        if t>=next_plot:
            ##############static profile plot#################
            plt.figure(fignum2+1, figsize=(8,7))
            plt.plot(x, z, linewidth = 3, label="profile, t = "+str(round(t/1000))+"ky")
            if lithkp:
                plt.plot(x[K_idx_start:K_idx_end], z[K_idx_start:K_idx_end],fillstyle="none",markeredgecolor="k", marker=".",ls = " ")
            else:
                plt.scatter(x[kpp],z[kpp], marker='o', s = 50, c = "steelblue", zorder=99)

            #include dots for knickpoints
            plt.xlabel('downstream distance (m)',fontsize = 14)
            plt.ylabel('elevation (m)',fontsize = 14)
            plt.title("stream profiles through time", fontsize=14)
            plt.legend()
            plt.tight_layout()
            #########slope area plot##############
            plt.figure(fignum2, figsize=(8,7))
            plt.loglog(A,S,".",label="t = "+str(round(t/1000))+"ky")
            plt.title('Slope-Area',fontsize = 14)
            plt.xlabel(r'drainage area (m$^2$)')
            plt.ylabel('slope (m/m)')
            #plt.xlim([2e6, 2e8])
            plt.legend()
            plt.tight_layout()
            next_plot=next_plot+plot_int
    #create animation and save as .gif
    animation = camera.animate(interval=750)
    animation.save(title_str+'long_prof.gif', writer = 'pillow')    #writer = 'imagemagick'

    plt.figure(fignum)
    plt.close()
    plt.figure(fignum2)
    plt.grid()
    ax = plt.gca()
    ax.minorticks_on
    ax.grid(which='minor', linestyle=':', linewidth=0.5, color='black')
    plt.figure(fignum2+1)
    plt.grid()

### Set parameters
Set parameters for the model domain, time, and stream power incision model. Initalize slope and area arrays for plotting.

**Run this cell,** but you won't need to change anything in this code block.

In [ ]:
xmax=3e4

dx=100
x_crit = 300;     #distance from divide to begin fluvial simulation (distances too close to divides do not behave like rivers)
x=np.arange(x_crit,xmax+1,dx)
#below, these are indicies between which lithology is decreased with lithkp option set to True
K_idx_start = round(len(x)*0.6)
K_idx_end = round(len(x)*0.8)

n=1
ka=6.69    #from Hack's Law
h=1.67     #reciprocal of hack's exponent
concavity = 0.5    #note concavity can range from ~0.35 to 0.55
m_n = concavity
m=concavity *n

S = np.zeros(len(x))
A=ka*x**h    #drainage areas
dt=100

fignum = 1
fignum2 = 2

## Set values of erodibility and uplift

You can change values of erodibility and uplift in the cell below.

**Increase or decrease erodibility.** The inital value for bedrock erodibiltiy, $K$, is 5e-5. Values of $K$ for this model can range from 1e-6 to 7e-5.

**Uplift rate** The initial uplift rate is 0.1 mm/year. You can try increasing or decreasing rock uplift by a factor of 4 to 0.0004 m/yr. 

In [ ]:
#erosion/uplift
K=5e-5     #erodibility coefficient
Ui=1e-4    #uplift rate (m/yr)

## Set change in boundary condition
You should change this value.

**Note on surface** option  ({baselevel_fall, increase_uplift}) – change in boundary condition. 'baselevel_fall' results in 5 m of instantaneous base level fall. 'increase_uplift' causes uplift rate to double from inital set value.

In [ ]:
# one of these surface options must be set. You can use a # to comment the lines in or out.
surface = 'baselevel_fall'
#surface = 'increase_uplift'

if surface == 'baselevel_fall':
    bl = -5
    U=Ui
    title_str = "Base Level Fall"    
elif surface == 'increase_uplift':
    bl=0
    U=Ui*2
    title_str = "Increased Uplift"

## Create initial steady state profile
In this cell, we create a steady state longitudinal profile.

**A note on option lithkp** ({True, False}) – Whether a lithology is spatialy variable in the longitudinal profiles. True means that a section of less erodible lithology is included in the long profile. False means that the long profile has uniform lithology.

In [ ]:
###use the options below to change the type of boundary conditions to model###
lithkp = False
#lithkp = True

if lithkp:
    #create a lithological knickpoint
    Ki=np.ones(len(x))*K
    #below, a section of the profile is 2-3 times less erodible
    Ki[K_idx_start:K_idx_end]=K/3
    z0=(Ui/Ki)**(1/n)*ka**(-m/n)*(1-h*m/n)**(-1)*(xmax**(1-h*m/n)-x**(1-h*m/n))
    # below, building the steady state profile with different lithologys
    z0[K_idx_start:K_idx_end] = z0[K_idx_start:K_idx_end]-(z0[K_idx_end-1]-z0[K_idx_end])
    z0[:K_idx_start] = z0[:K_idx_start]+(z0[K_idx_start]-z0[K_idx_start-1])  
    title_str = "Lithological Knickpoint"
else:
    Ki=np.ones(len(x))*K
    z0=(Ui/K)**(1/n)*ka**(-m/n)*(1-h*m/n)**(-1)*(xmax**(1-h*m/n)-x**(1-h*m/n))
z=z0.copy()       #elevations to change

### Plotting inital values before change in boundary conditions
Plot the slope and area data at each point on the landscape (in log-log space) and the inital longitudinal profile.

In [ ]:
#slope area plotting
S[:-1]=-np.diff(z)/dx
S[-1]= S[-2]    # make the last element of slope equal to the second-to-last element of slope
plt.figure(1, figsize = (6,5))
plt.loglog(A,S, 'k.',label="t = 0 ky")
if lithkp:
    plt.loglog(A[K_idx_start:K_idx_end], S[K_idx_start:K_idx_end], color="orange",marker=".",ls = " ", label="less erodible")
plt.title('Slope-Area',fontsize = 14)
plt.xlabel(r'drainage area (m$^2$)',fontsize = 14)
plt.ylabel('slope',fontsize = 14)
plt.grid()
ax = plt.gca()
ax.minorticks_on
ax.grid(which='minor', linestyle=':', linewidth=0.5, color='black')

plt.figure(2, figsize=(6,5))
plt.plot(x, z, marker=".",linewidth = 3, label="profile, t = 0ky")
if lithkp:
    plt.plot(x[K_idx_start:K_idx_end], z[K_idx_start:K_idx_end], marker=".",color="orange",ls = " ", label="less erodible")

plt.xlabel('distance from source (m)',fontsize = 14)
plt.ylabel('elevation (m)',fontsize = 14)
plt.title("inital longitudinal profile", fontsize=14)
plt.grid()
plt.legend()
plt.tight_layout()


### Set model run time
`modelruntime` is the number of years the model will run. You can play around with changing this value.
Time to steady state will depend on selected values of $K$ and uplift rate. When K = 5e-5, the model will reach steady state before 150,000 years; you don't need to run it any longer than that. 

`num_outputs` is the number of intermediate time steps you want to plot on the output figures. Setting `num_outputs` to 5 - 10 works well, but you can experiment with more outputs.

**Finally, run the model through time** by running the cell below.

In [ ]:
# Change the values of modelruntime and num_outputs
# run this cell to run the model through time
modelruntime = 50000    #model run time in years
num_outputs = 5

runmodel(tmax=modelruntime, z=z, num_plots=num_outputs)

**The cell below** displays an animated gif of knickpoint retreat through time that was created in the model run.

In [ ]:
from IPython.display import IFrame
IFrame(title_str+'long_prof.gif', width=1000, height=900)

## Working with the model: Part 1
**Rock erodibility**: This parameter is analogous to the erodibility parameter (K) we discussed in lecture. Hard rocks have low erodibility, weak rocks have high erodibility. 

•	Run the simulation using the parametewn bel:

$K$ = 7e-5 yr$^{-1}$

$U$ = 0.0001 m/yr

`lithkp` = False

`surface` = baselevel_fall

`modelruntime` = 50,000.
1. How far does the knickpoint migrate upstream during the first 50,000 years of the model simulation? Give your answer in m.
2. What is the average migration rate of the knickpoint over 50,000 years? Give your answer in cm/year.

* Now change $K$ to 7e-6 yr$^{-1}$ and rerun the model for 50,000.
3. How far does the knickpoint migrate upstream during the first 50,000 years of the model simulation? Give your answer in m.
4. What is the average migration rate of the knickpoint over 50,000 years? Give your answer in cm/year.
5. How does the migration rate compare between the low erodibility and high erodibility model runs? What does this tell us about the time scale of adjustment for hard rocks vs. soft rocks?
6. Give two examples of rocks/channel material with low erodibility
   
8.	Give two examples of rocks/channel material with high erodibiity.

 


## Working with the model: Part 2
**Increased uplift**
Run the simulation using the parameters below:

$K$ = 5e-5 yr$^{-1}$

$U$ = 0.0001 m/yr

`lithkp` = False

`surface` = uplift_increase

`modelruntime` = 50,000

Use the slope-area plots and the plots of the channel profile through time to consider the following questions.

1. Looking at the channel profiles through time, qualitatively describe what happens to the longitudinal profile following an increase in uplift rate. How is the new steady state channel different from the initial channel?
2. Now looking at the slope area figure through time. Part of the data points in this figures have reached a new steady state and some of them have not. Which section of the figure has data that has reached new steady state. How do you know?
3. If you think about how the slope area plot will change through time, what will the slope area plot look like when the entire channel profile has reached its new steady state with higher uplift rate? (describe or sketch)

## Working with the model: Part 3
**Lithological knickpoints**

Run the simulation using the parameters below:

$K$ = 5e-5 yr$^{-1}$

$U$ = 0.0001 m/yr

`lithkp` = True

`surface` = baselevel_fall

`modelruntime` = 50,000


Use the slope-area and channel profile figures to consider the following questions.

1. Looking at the **inital** figures of  slope-area and channel profile (figures before base level fall). Is there a knickpoint? How can you tell from the channel profile and slope area figure? Is this channel in steady state?
2. **Challenge:** Run the model through time and change the value of the parameter `num_outputs` estimate knickpoint migration rate in the profile sections with different erodibilties.
   * What is the knickpoint migration rate in the more erodible sections?
   * What is the knickpoint migration rate in the less erodible section?